In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow(folder='logFolder'):
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./{}'.format(folder), now)
    return nowFolder

In [3]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))

### Loading a previous graph

In [3]:
tf.reset_default_graph()
savePath = './models/2019-02-11--18-58-58/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [4]:
summaryFolder = getNow()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    writer.close()
    
print('tensorboard --logdir={}'.format( summaryFolder ))

tensorboard --logdir=./logFolder/2018-06-28--17-37-19


### Without loading preivous state

In [2]:
tf.reset_default_graph()
savePath = './models/2018-06-28--15-49-55/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [12]:
with tf.Session() as sess:
    sess.run('misc/init')
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})

print(errVal)

3.48578


### Run the optimizer

In [15]:
with tf.Session() as sess:
    sess.run('misc/init')
    for i in range(1000):
        sess.run( 'optimizers/Adam', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})
        
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})

print(errVal)

0.000823804


### After loading previous state

In [16]:
tf.reset_default_graph()
savePath = './models/2018-06-28--15-49-55/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [18]:
with tf.Session() as sess:
    sess.run('misc/init')
    saver.restore(sess, savePath)
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})
    
print(errVal)

INFO:tensorflow:Restoring parameters from ./models/2018-06-28--15-49-55/model.ckpt
0.000828783


### Get variables

In [26]:
tf.reset_default_graph()
savePath = './models/2018-06-28--15-49-55/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [28]:
graph = tf.get_default_graph()
err   = graph.get_tensor_by_name('model/err:0')
inp   = graph.get_tensor_by_name('io/inp:0')
out   = graph.get_tensor_by_name('io/out:0')

with tf.Session() as sess:
    sess.run('misc/init')
    saver.restore(sess, savePath)
    errVal = sess.run( err, feed_dict={
            inp : X, out : y})
    
print(errVal)

INFO:tensorflow:Restoring parameters from ./models/2018-06-28--15-49-55/model.ckpt
0.000828783


### Using the graph collections

In [20]:
graph = tf.get_default_graph()
variables = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [21]:
[v.name  for v in variables]

['model/layer_00000/W:0',
 'model/layer_00000/b:0',
 'model/layer_00001/W:0',
 'model/layer_00001/b:0',
 'model/layer_00002/W:0',
 'model/layer_00002/b:0']

In [22]:
[v.name for v in graph.get_collection(tf.GraphKeys.VARIABLES)]

['model/layer_00000/W:0',
 'model/layer_00000/b:0',
 'model/layer_00001/W:0',
 'model/layer_00001/b:0',
 'model/layer_00002/W:0',
 'model/layer_00002/b:0',
 'optimizers/beta1_power:0',
 'optimizers/beta2_power:0',
 'optimizers/model/layer_00000/W/Adam:0',
 'optimizers/model/layer_00000/W/Adam_1:0',
 'optimizers/model/layer_00000/b/Adam:0',
 'optimizers/model/layer_00000/b/Adam_1:0',
 'optimizers/model/layer_00001/W/Adam:0',
 'optimizers/model/layer_00001/W/Adam_1:0',
 'optimizers/model/layer_00001/b/Adam:0',
 'optimizers/model/layer_00001/b/Adam_1:0',
 'optimizers/model/layer_00002/W/Adam:0',
 'optimizers/model/layer_00002/W/Adam_1:0',
 'optimizers/model/layer_00002/b/Adam:0',
 'optimizers/model/layer_00002/b/Adam_1:0']

### Getting the variable values

In [25]:
with tf.Session() as sess:
    saver.restore(sess, savePath)
    for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print(v.name)
        print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./models/2018-06-28--15-49-55/model.ckpt
model/layer_00000/W:0
[[ 0.5717333   0.19548211  0.45528662  0.62857431  0.22678171 -0.18899886
   0.66012752 -0.1440849   0.15550865  0.60070246]
 [ 0.86717474  0.49768844  0.73439205 -0.01208313  0.17761439  0.49848092
   0.91902369  0.16994558  0.18100658  0.00521727]]
model/layer_00000/b:0
[[ 0.33378291  0.34686857  0.34494224  0.33497506  0.34497482  0.37368101
   0.33496913  0.37542731  0.34816936  0.31579486]]
model/layer_00001/W:0
[[ 0.29121259 -0.02237164  0.86239558  0.39535969  0.02165305]
 [-0.00696487  0.29616392  0.37376603  0.65181971  0.08567395]
 [ 0.17245257  0.06465442  0.80565006  0.4744812   0.39250892]
 [ 0.69784826  0.81502718  0.10170063  0.45668015  0.42934233]
 [ 0.49389875  0.1705883   0.53318369  0.2263654   0.45932907]
 [ 0.72302705  0.68155563  0.55588037  0.3135342   0.61946237]
 [ 0.18898198  0.48440674  0.68116242  0.01322642  0.33675501]
 [ 0.68009257  0.85164595  1.1441

## Transfer Learning

In [4]:
tf.reset_default_graph()
savePath = './models/2018-06-28--15-49-55/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [5]:
graph = tf.get_default_graph()
err   = graph.get_tensor_by_name('model/err:0')
inp   = graph.get_tensor_by_name('io/inp:0')
out   = graph.get_tensor_by_name('io/out:0')

In [6]:
for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v.name)

model/layer_00000/W:0
model/layer_00000/b:0
model/layer_00001/W:0
model/layer_00001/b:0
model/layer_00002/W:0
model/layer_00002/b:0


In [7]:
varsToTrain = [v for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if 'model/layer_00002' in v.name]
with tf.variable_scope('new_opt'):
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(err, var_list=varsToTrain)
    
with tf.variable_scope('new_misc'):
    init  = tf.global_variables_initializer()

In [9]:
summaryFolder = getNow()

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, savePath)
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    
    for i in range(1000):
        sess.run(opt, feed_dict={inp: X, out: y})
        
    for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print('-----[{:30s}]--------'.format(v.name))
        print(v.eval())
    
    writer.close()
    
print('tensorboard --logdir={}'.format( summaryFolder ))

INFO:tensorflow:Restoring parameters from ./models/2018-06-28--15-49-55/model.ckpt
-----[model/layer_00000/W:0         ]--------
[[ 0.5717333   0.19548211  0.45528662  0.62857431  0.22678171 -0.18899886
   0.66012752 -0.1440849   0.15550865  0.60070246]
 [ 0.86717474  0.49768844  0.73439205 -0.01208313  0.17761439  0.49848092
   0.91902369  0.16994558  0.18100658  0.00521727]]
-----[model/layer_00000/b:0         ]--------
[[ 0.33378291  0.34686857  0.34494224  0.33497506  0.34497482  0.37368101
   0.33496913  0.37542731  0.34816936  0.31579486]]
-----[model/layer_00001/W:0         ]--------
[[ 0.29121259 -0.02237164  0.86239558  0.39535969  0.02165305]
 [-0.00696487  0.29616392  0.37376603  0.65181971  0.08567395]
 [ 0.17245257  0.06465442  0.80565006  0.4744812   0.39250892]
 [ 0.69784826  0.81502718  0.10170063  0.45668015  0.42934233]
 [ 0.49389875  0.1705883   0.53318369  0.2263654   0.45932907]
 [ 0.72302705  0.68155563  0.55588037  0.3135342   0.61946237]
 [ 0.18898198  0.4844067